In [51]:
import numpy as np
import pickle
import torch
import sys
import trajnetplusplustools
from tqdm import tqdm

## To run prediction:

1. clone trajnet++ baselines code [repo](https://github.com/vita-epfl/trajnetplusplusbaselines)
2. install [trajnetplusplustools](https://github.com/vita-epfl/trajnetplusplustools): clone and then ```python setup.py install```
3. download trained model file from [drive](https://drive.google.com/file/d/1wHj0QCGBLjxh5pVD3-KvlhrIPVCWMI7O/view?usp=share_link)  to ```prediction/trained_model/social_lstm/``` 

## Load Model

In [27]:
sys.path.append("../pedestriandataset") #change to where the trajnetplusplusbaselines folder is

from trajnetplusplusbaselines.trajnetbaselines.lstm.gridbased_pooling import GridBasedPooling
from trajnetplusplusbaselines.trajnetbaselines.lstm.lstm import LSTM, LSTMPredictor, drop_distant

pool = GridBasedPooling(type_='social', hidden_dim=128,
                        cell_side=0.6, n=16, front=True,
                        out_dim=256, embedding_arch='two_layer',
                        constant=0, pretrained_pool_encoder=None,
                        norm=0, layer_dims=[1024], latent_dim=16)

model = LSTM(pool=pool,
             embedding_dim=64,
             hidden_dim=128,
             goal_flag=False,
             goal_dim=0)


In [28]:
model_path = 'prediction/trained_model/social_lstm/lstm_social_None.pkl.epoch25.state'
#sys.path.append("./trajnetplusplusbaselines/trajnetbaselines")

with open(model_path, 'rb') as f:
    checkpoint = torch.load(f)
    
pretrained_state_dict = checkpoint['state_dict']

model.load_state_dict(pretrained_state_dict, strict=False)

<All keys matched successfully>

## Load test data

In [34]:
ls datasets/pedestrian/trajdata/test

biwi_hotel.ndjson          crowds_zara01.ndjson
cff_10.ndjson              crowds_zara03.ndjson
cff_17.ndjson              lcas.ndjson
crowds_students001.ndjson  wildtrack.ndjson
crowds_students003.ndjson


In [38]:
file = 'biwi_hotel.ndjson'
fullpath  = 'datasets/pedestrian/trajdata/test/' + file

reader = trajnetplusplustools.Reader(fullpath, scene_type='paths')

In [39]:
scenes = [(file, s_id, s) for s_id, s in reader.scenes()]
len(scenes)

40

In [40]:
scene1 = trajnetplusplustools.Reader.paths_to_xy(scenes[0][2])
scene1 = torch.Tensor(scene1)
scene1.shape

torch.Size([21, 8, 2])

In [46]:
def recover_pos(last_obs, delta):
    output = [last_obs]
    for i, d in enumerate(delta):
        output.append(output[i]+d[...,:2])
        
    return torch.stack(output)
    
def predict_one(scene):
    scene1 = trajnetplusplustools.Reader.paths_to_xy(scene[2])
    scene1 = torch.Tensor(scene1)

    seq_length = 21
    obs_length = 9
    pred_length = 12
    batch_scene_goal = []
    batch_split = [0]
    batch_split.append(int(scene1.shape[1]))
    batch_split = torch.Tensor(batch_split).long()

    observed = scene1[:obs_length]
    prediction_truth = scene1[obs_length:seq_length-1]
    targets = scene1[obs_length:seq_length] - scene1[obs_length-1:seq_length-1]

    with torch.no_grad():
        rel_outputs, _ = model(observed, batch_scene_goal, batch_split, prediction_truth)
        
    delta = rel_outputs[-12:]
    out = recover_pos(observed[-1], delta)
    uq = delta[..., 2:]
    
    return scene1, out, uq

In [47]:
scene1, out, uq = predict_one(scenes[0])

In [52]:
results = []
for s in tqdm(scenes):
    gt, out, uq = predict_one(s)
    results.append((gt.detach().numpy(), out.detach().numpy(), uq.detach().numpy()))

100%|█████████████████████████████████████████████████| 40/40 [00:01<00:00, 22.31it/s]


In [54]:
!ls outputs/

pedestrian_sociallstm_out.pkl


In [55]:
with open('outputs/biwi_sociallstm_uq.pkl', 'wb') as f:
    pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)


In [56]:
all_test = ! ls datasets/pedestrian/trajdata/test

In [58]:
all_test = all_test[:-1]
all_test

['biwi_hotel.ndjson',
 'cff_10.ndjson',
 'cff_17.ndjson',
 'crowds_students001.ndjson',
 'crowds_students003.ndjson',
 'crowds_zara01.ndjson',
 'crowds_zara03.ndjson',
 'lcas.ndjson']

In [59]:
scenes = []

for file in all_test:

    fullpath  = 'datasets/pedestrian/trajdata/test/' + file

    reader = trajnetplusplustools.Reader(fullpath, scene_type='paths')

    scenes += [(file, s_id, s) for s_id, s in reader.scenes()]

In [60]:
results = []
for s in tqdm(scenes):
    gt, out, uq = predict_one(s)
    results.append((gt.detach().numpy(), out.detach().numpy(), uq.detach().numpy()))

100%|███████████████████████████████████████| 6409/6409 [12:51<00:00,  8.31it/s]


In [61]:
with open('outputs/sociallstm_all_with_uq.pkl', 'wb') as f:
    pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

# uniform data freequency

In [6]:

import ndjson

In [2]:
test_files = !ls datasets/pedestrian/trajdata/test

In [3]:
test_files

['biwi_hotel.ndjson',
 'cff_10.ndjson',
 'cff_17.ndjson',
 'crowds_students001.ndjson',
 'crowds_students003.ndjson',
 'crowds_zara01.ndjson',
 'crowds_zara03.ndjson',
 'lcas.ndjson',
 'wildtrack.ndjson']

In [7]:
for file in test_files:
    with open('datasets/pedestrian/trajdata/test/' + file) as f:
        data = ndjson.load(f)
        print(data[0])

{'scene': {'id': 6, 'p': 25, 's': 500, 'e': 700, 'fps': 2.5, 'tag': [4, []]}}
{'scene': {'id': 3, 'p': 3, 's': 1072, 'e': 1152, 'fps': 2.5, 'tag': [3, [4]]}}
{'scene': {'id': 1, 'p': 5, 's': 356, 'e': 436, 'fps': 2.5, 'tag': [4, []]}}
{'scene': {'id': 2, 'p': 0, 's': 50, 'e': 250, 'fps': 2.5, 'tag': [2, []]}}
{'scene': {'id': 1, 'p': 2, 's': 30, 'e': 230, 'fps': 2.5, 'tag': [3, [4]]}}
{'scene': {'id': 6, 'p': 2, 's': 10, 'e': 210, 'fps': 2.5, 'tag': [2, []]}}
{'scene': {'id': 0, 'p': 0, 's': 10, 'e': 210, 'fps': 2.5, 'tag': [3, [3]]}}
{'scene': {'id': 2, 'p': 588, 's': 34, 'e': 54, 'fps': 2.5, 'tag': [1, []]}}
{'scene': {'id': 1, 'p': 5, 's': 10, 'e': 110, 'fps': 2, 'tag': [3, [4]]}}


In [11]:
count = 0
for i in data:
    if 'scene' in i.keys():
        count +=1

In [12]:
count

266